# 5. 交叉编码器重排序
本节课，将使用交叉编码器重排序的技术，对检索到的结果进行相关性分析。重排序是一种根据结果与特定查询的相关性来排序和评分的方法。


<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一底层原理" data-toc-modified-id="一、底层原理">一、底层原理</a></span></li>
        <li>
        <span><a href="#二实现过程" data-toc-modified-id="二、实现过程">二、实现过程</a></span></li><li>
        <ul class="toc-item">
            <li><span><a href="#21-导入辅助函数" data-toc-modified-id="2.1 导入辅助函数">2.1 导入辅助函数</a></span></li>
            <li><span><a href="#22-长尾部分的重排序" data-toc-modified-id="2.2 长尾部分的重排序">2.2 长尾部分的重排序</a></span></li>
            <li><span><a href="#23-结合查询扩展的重排序" data-toc-modified-id="2.3 结合查询扩展的重排序">2.3 结合查询扩展的重排序</a></span></li>
        </ul>
        </li>
    </ul>
</div>

## 一、底层原理
在重排序过程中，在得到特定查询检索到结果之后，需要将这些结果连同查询一起传递给一个重排序模型。这可以重新排列输出，使最相关的结果具有最高的排名。另一种思考方式是，重排序模型根据查询对每个结果打分，得分最高的就是最相关的结果。最后，以选择排名最高的结果作为与特定查询最相关的结果。
 
## 二、实现过程
### 2.1 导入辅助函数

In [1]:
# 导入辅助函数并把数据加载到Chroma中 
from helper_utils import load_chroma, word_wrap, project_embeddings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import numpy as np

In [3]:
# 使用代理可能出现网络问题，将以下端口号1080全部替换成自己的vpn的端口号
import os
os.environ['HTTPS_PROXY']='http://127.0.0.1:1080'
os.environ["HTTP_PROXY"]='http://127.0.0.1:1080'

embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = load_chroma(filename='./data/microsoft_annual_report_2022.pdf', collection_name='microsoft_annual_report_2022', embedding_function=embedding_function)
chroma_collection.count()

349

### 2.2 长尾部分的重排序

In [4]:
# 之前一般设定返回5个结果，现在要求返回10个结果，加入了部分可能有用的的长尾结果
query = "What has been tchhe investment in research and development?"
results = chroma_collection.query(query_texts=query, n_results=10, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

• operating expenses increased $ 1. 5 billion or 14 % driven by
investments in gaming, search and news advertising, and windows
marketing. operating expenses research and development ( in millions,
except percentages ) 2022 2021 percentage change research and
development $ 24, 512 $ 20, 716 18 % as a percent of revenue 12 % 12 %
0ppt research and development expenses include payroll, employee
benefits, stock - based compensation expense, and other headcount -
related expenses associated with product development. research and
development expenses also include third - party development and
programming costs, localization costs incurred to translate software
for international markets, and the amortization of purchased software
code and services content. research and development expenses increased
$ 3. 8 billion or 18 % driven by investments in cloud engineering,
gaming, and linkedin. sales and marketing

also increased the number of identified partners in the black partner
growth initiati

In [5]:
# BERT交叉编码器同时渠道查询和文档，通过一个分类器传递，获得一个得分
# 利用该得分作为检索结果的相关性或排名的得分
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [6]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print("Scores:")
for score in scores:
    print(score)

Scores:
-3.2845027
-9.440044
-11.208979
-11.164228
-9.3616
-11.032126
-10.818983
-11.283258
-11.276759
-11.241401


In [7]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o+1)

New Ordering:
1
5
2
7
6
4
3
10
9
8


### 2.3 结合查询扩展的重排序

In [8]:
# 接下来把之前获得的结果排序前5名传递给LLM
original_query = "What were the most important factors that contributed to increases in revenue?"
generated_queries = [
    "What were the major drivers of revenue growth?",
    "Were there any new product launches that contributed to the increase in revenue?",
    "Did any changes in pricing or promotions impact the revenue growth?",
    "What were the key market trends that facilitated the increase in revenue?",
    "Did any acquisitions or partnerships contribute to the revenue growth?"
]

In [9]:
queries = [original_query] + generated_queries

results = chroma_collection.query(query_texts=queries, n_results=10, include=['documents', 'embeddings'])
retrieved_documents = results['documents']

In [10]:
# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

unique_documents = list(unique_documents)

In [11]:
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])

In [12]:
scores = cross_encoder.predict(pairs)

In [13]:
print("Scores:")
for score in scores:
    print(score)

Scores:
-4.8184843
-10.148884
-10.042843
-9.8078785
-10.711212
-4.6518908
-10.000139
-11.079269
-5.141833
-3.768155
-7.754099
-9.768024
-6.9020905
-7.9171762
-9.918427
-9.357723
-10.083939
-1.1369972
-4.3417673
-8.505105
-7.490654
-5.2747474
-3.7948644


In [14]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o)

New Ordering:
17
9
22
18
5
0
8
21
12
20
10
13
19
15
11
3
14
6
2
16
1
4
7
